# read_block()

The `read_block()` function is the basic interface for reading block-level ECOG data from the UCSF Chang lab. It has one required parameter that defines the base data directory `datadir`. By default `read_block()` will read data from the 'ecogDS' subdirectory of `datadir`.

In [1]:
from ecog_ucsf.block import read_block

datadir = '../../ecog/datasets/GP31_B40'
ds = read_block(datadir)
ds

ECBlock(datadir='../../ecog/datasets/GP31_B40', subdir='ecogDS', data=[[ 130.33782959  247.03282166  217.74197388 ...,  227.62039185
   253.04103088  129.75482178]
 [ 110.74650574  211.14225769  192.89465332 ...,  175.30302429
   186.76789856   94.83296204]
 [  99.78131866  188.86553955  169.49157715 ...,  184.86541748
   199.61947632  101.62522888]
 ..., 
 [ 155.18597412  291.36831665  249.77731323 ...,  325.25637817
   361.01947021  180.72859192]
 [-259.61843872 -491.78964233 -423.64752197 ..., -323.57369995
  -382.38088989 -196.47114563]
 [          nan           nan           nan ...,           nan
            nan           nan]], htkrate=4000.0, badchan=[256, 113, 114, 115, 116, 124, 128])

`read_block()` returns an ECBlock object, which contains the block data as a numpy ndarray and associated metadata. The `data` attribute holds the ndarray. The first axis of this array is the electrode channels, and the second axis is sample 'times' in the form of an integer index.

In [2]:
ds.data.shape

(256, 288801)

Other attributes you can read include the sample rate (converted to Hz) reported by the `.htk` files, the list of bad channels in the block, and the block's data directory and subdirectory.

In [3]:
print(ds.htkrate)
print(ds.badchan)
print(ds.datadir)
print(ds.subdir)

4000.0
[256, 113, 114, 115, 116, 124, 128]
../../ecog/datasets/GP31_B40
ecogDS


## The `subdir` parameter

Use the `subdir` parameter to read from a non-default subdirectory of `datadir`. The `.htk` files in `subdir` may contain data that is a different shape than is found in the default 'ecogDS' subdirectory.

In [4]:
subdir = 'HilbAA_70to150_8band'
bands = read_block(datadir, subdir)
bands

ECBlock(datadir='../../ecog/datasets/GP31_B40', subdir='ecogDS', data=[[[  4.22455025   5.36758852   5.80154228 ...,   4.39262581   5.51103258
     4.25414467]
  [  4.2146287    5.24366951   5.78416252 ...,   4.44086552   5.42778873
     4.1641655 ]
  [  4.18704987   5.09346533   5.7275176  ...,   4.4530158    5.30415964
     4.02637386]
  ..., 
  [  4.15087366   5.57430077   5.61382198 ...,   4.02628422   5.50698853
     4.20953608]
  [  4.1921711    5.53314447   5.71599436 ...,   4.18471861   5.55061865
     4.27777195]
  [  4.21693707   5.46424341   5.77882814 ...,   4.30722809   5.55216503
     4.29264069]]

 [[  3.09665585   3.61621547   3.19286013 ...,   2.23635387   2.08436179
     3.25961304]
  [  3.07151604   3.57516837   3.20979118 ...,   2.28917551   2.01924372
     3.29403639]
  [  3.02797365   3.51849747   3.20194244 ...,   2.31910515   1.93104589
     3.30092835]
  ..., 
  [  3.05056548   3.64777827   2.98912096 ...,   1.94276071   2.13781428
     3.0026288 ]
  [  3.08691

The `.htk` files in 'HilbAA_70to150_8band' contain 2D data, with sample values in 8 frequency bands. The axes of the output ndarray data are now (channels, sample_times, bands).

In [5]:
bands.data.shape

(256, 288801, 8)

## The `channel_cb` parameter.

The `channel_cb` parameter may be used to apply a callback function to the data read from the `.htk` files on a per-channel basis. This function may change the shape of the input data, as long as the output shapes of all the channels are compatible with each other and can be compiled into a single output ndarray.

As an example, we'll use `decimate()` to downsample the signals from the `ecogDS` subdirectory. `read_block()` provides the channel data (a numpy ndarray) as the input to the callback function. In the example a lambda function calls `decimate()` on this ndarray and supplies additional parameters `q`, `axis`, and `zero_phase`.

In [6]:
from scipy.signal import decimate
dec = read_block(
    datadir,
    channel_cb=lambda chan: decimate(chan, q=10, axis=0, zero_phase=True)
)
dec.data.shape

(256, 28881)

The result is the same as the `ds` output from before, except the length of the second axis has been reduced by a factor of 10.

### Three notes about `channel_cb`

There are several important points to observe if you use the `channel_cb` callback parameter:

1. The callback applies separately to each channel as the corresponding `.htk` file is read. It does not apply to the output ndarray that contains all of the channels.
1. The axis index used in the callback refers to the axis location in the *input* ndarray of a single channel, not the location in the *output* ndarray of all channels. The `axis` parameter in the preceding example illustrates this point. The input ndarrays for each channel are 1D and therefore `decimate()` must operate on axis 0. But in the output array, axis 1 is the axis that shows the effect of downsampling.
1. The `htkrate` attribute reports the sample rate found in the input `.htk` files. If you downsample your data in the callback, then the effective sample rate will not match `htkrate`.


### Reducing dimensionality

The `channel_cb` callback can also reduce the dimensionality of the input arrays. For example, numpy's `mean()` method can reduce the 2D data found in the 'HilbAA_70to150_8band' subdirectory. As we saw earlier in `bands`, the first axis of each channel corresponds to sample times and the second corresponds to frequency bands (these correspond to axis 1 and axis 2 of `bands.data`, respectively; axis 0 of `bands.data` corresponds to channels).

For each *input* channel the sample times are contained in axis 0 and the frequency bands are in axis 1. If we take the mean along axis 1, then the result is an output ndarray with axes (channels, sample_times) where the values are the mean value of the 8 frequency bands per-channel and per-sample_time.

In [7]:
mn = read_block(
    datadir,
    subdir,
    channel_cb=lambda chan: chan.mean(axis=1)
)
mn.data.shape

(256, 288801)

It's also possible to reduce the dimensionality of a dataset and reduce the length of a dimension in a single callback. Here the dimensionality is reduced by `mean()` and the length by `decimate()`.

In [8]:
from scipy.signal import decimate
r2 = read_block(
    datadir,
    subdir,
    channel_cb=
        lambda chan: decimate(
            chan.mean(axis=1), q=10, axis=0, zero_phase=True
        )
)
r2.data.shape

(256, 28881)

Additional calculations on the channel data can also be done. The final example calculates zscores of the downsampled mean data.

In [9]:
from scipy.stats import zscore
z = read_block(
    datadir,
    subdir,
    channel_cb=
        # mean of bands -> downsample -> compute zscores
        lambda chan: zscore(
            decimate(
                chan.mean(axis=1), q=10, axis=0, zero_phase=True
            )
        )
)
z.data.shape

(256, 28881)